<a href="https://colab.research.google.com/github/AUT-Student/SML-Project/blob/main/SML_Project_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: Statistical Machine Learning
<br>
<b>Description:</b> Project | Question 2
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Dataset

In [2]:
!gdown 1mFilhzDbZqb4xC6ifBLPjha1Svmqcr9X

Downloading...
From: https://drive.google.com/uc?id=1mFilhzDbZqb4xC6ifBLPjha1Svmqcr9X
To: /content/SML_Project_Dataset.zip
100% 436k/436k [00:00<00:00, 115MB/s]


In [3]:
!unzip /content/SML_Project_Dataset.zip

Archive:  /content/SML_Project_Dataset.zip
  inflating: Gold.csv                
  inflating: Kekhak.csv              
  inflating: Khodro.csv              
  inflating: Overall.csv             
  inflating: Shasta.csv              
  inflating: Atlas.csv               
  inflating: Dollar.csv              
  inflating: Foolad.csv              


In [4]:
overall_dataset = pd.read_csv("/content/Overall.csv")
dolar_dataset = pd.read_csv("/content/Dollar.csv")
gold_dataset = pd.read_csv("/content/Gold.csv")

In [6]:
overall_dataset["<DTYYYYMMDD>"] = overall_dataset["Date"].apply(lambda x:int(x[:4]+x[5:7]+x[8:10]))
dolar_dataset["<DTYYYYMMDD>"] = dolar_dataset["Date"].apply(lambda x:int(x[:4]+x[5:7]+x[8:10]))
gold_dataset["<DTYYYYMMDD>"] = gold_dataset["Date"].apply(lambda x:int(x[:4]+x[5:7]+x[8:10]))

In [7]:
atlas_dataset = pd.read_csv("/content/Atlas.csv").sort_values("<DTYYYYMMDD>")
foolad_dataset = pd.read_csv("/content/Foolad.csv").sort_values("<DTYYYYMMDD>")
kekhak_dataset = pd.read_csv("/content/Kekhak.csv").sort_values("<DTYYYYMMDD>")
khodro_dataset = pd.read_csv("/content/Khodro.csv").sort_values("<DTYYYYMMDD>")
shasta_dataset = pd.read_csv("/content/Shasta.csv").sort_values("<DTYYYYMMDD>")

In [8]:
datasets = {
    "Atlas": atlas_dataset,
    "Foolad": foolad_dataset,
    "Kekhak": kekhak_dataset,
    "Khodro": khodro_dataset,
    "Shasta": shasta_dataset
}

# Return

In [9]:
overall_dataset["Return"] = overall_dataset["Close"].diff()/overall_dataset["Close"]
dolar_dataset["Return"] = dolar_dataset["Close"].diff()/dolar_dataset["Close"]
gold_dataset["Return"] = gold_dataset["Close"].diff()/gold_dataset["Close"]

In [10]:
overall_dataset = overall_dataset[["<DTYYYYMMDD>", "Return"]]
dolar_dataset = dolar_dataset[["<DTYYYYMMDD>", "Return"]]
gold_dataset = gold_dataset[["<DTYYYYMMDD>", "Return"]]

In [11]:
for name in datasets:
  dataset = datasets[name]
  dataset = dataset[(dataset["<DTYYYYMMDD>"] >= 20210101) & (dataset["<DTYYYYMMDD>"] <= 20221230)]
  dataset["Return"] = dataset["<CLOSE>"].diff()/dataset["<CLOSE>"]  
  datasets[name] = dataset

<ipython-input-11-ece0a09dd663>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Return"] = dataset["<CLOSE>"].diff()/dataset["<CLOSE>"]


# Autocorrelation

In [12]:
for lag in range(1, 6):
  autocorrelation = overall_dataset["Return"].autocorr(lag=lag)
  print(f"Lag = {lag}, Autocorrelation = {autocorrelation}")

Lag = 1, Autocorrelation = 0.3420962070256291
Lag = 2, Autocorrelation = 0.026094104901236134
Lag = 3, Autocorrelation = 0.06856595465315196
Lag = 4, Autocorrelation = -0.008159864385430154
Lag = 5, Autocorrelation = -0.029116582025897918


# Correlation

In [13]:
# Corrlation with stock shares
for name in datasets:
  common_dataset = datasets[name][["<DTYYYYMMDD>", "Return"]]\
                  .set_index("<DTYYYYMMDD>")\
                  .join(overall_dataset\
                        .set_index("<DTYYYYMMDD>"),
                        how="inner", lsuffix='_share', rsuffix='_overall')\
                  .dropna()

  correlation = common_dataset["Return_share"].corr(common_dataset["Return_overall"])
  print(f"{name}: {correlation}")

Atlas: 0.85945676743588
Foolad: 0.3087456241782305
Kekhak: 0.14459757499068746
Khodro: 0.6678754557875755
Shasta: 0.03573243689880025


In [14]:
# Correlation with Dollar
common_dataset = dolar_dataset\
                .set_index("<DTYYYYMMDD>")\
                .join(overall_dataset\
                      .set_index("<DTYYYYMMDD>"),
                      how="inner", lsuffix='_dollar', rsuffix='_overall')\
                .dropna()

correlation = common_dataset["Return_dollar"].corr(common_dataset["Return_overall"])
print(f"Dollar: {correlation}")

Dollar: 0.18703521424717875


In [15]:
# Correlation with Gold
common_dataset = gold_dataset\
                .set_index("<DTYYYYMMDD>")\
                .join(overall_dataset\
                      .set_index("<DTYYYYMMDD>"),
                      how="inner", lsuffix='_gold', rsuffix='_overall')\
                .dropna()

correlation = common_dataset["Return_gold"].corr(common_dataset["Return_overall"])
print(f"Gold: {correlation}")

Gold: 0.2706968021626852
